<a href="https://colab.research.google.com/github/j143/notebooks/blob/main/systemds_dev_standalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Developer notebook for Apache SystemDS

#### Install Java
This installs Java 8 of Open JDK

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#### Set Environment Variables
Set the locations where Spark and Java are installed.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
# os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


## Apache SystemDS

#### Setup

In [1]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')


#### Install Apache Maven

In [2]:
import os

# Download the maven source.
maven_version = 'apache-maven-3.9.6'
maven_path = f"/opt/{maven_version}"
if not os.path.exists(maven_path):
  run(f"wget -q -nc -O apache-maven.zip https://downloads.apache.org/maven/maven-3/3.9.6/binaries/{maven_version}-bin.zip")
  run('unzip -q -d /opt apache-maven.zip')
  run('rm -f apache-maven.zip')

# Let's choose the absolute path instead of $PATH environment variable.
def maven(args):
  run(f"{maven_path}/bin/mvn {args}")

maven('-v')


>> wget -q -nc -O apache-maven.zip https://downloads.apache.org/maven/maven-3/3.9.6/binaries/apache-maven-3.9.6-bin.zip

>> unzip -q -d /opt apache-maven.zip

>> rm -f apache-maven.zip

>> /opt/apache-maven-3.9.6/bin/mvn -v
Apache Maven 3.9.6 (bc0240f3c744dd6b6ec2920b3cd08dcc295161ae)
Maven home: /opt/apache-maven-3.9.6
Java version: 11.0.21, vendor: Ubuntu, runtime: /usr/lib/jvm/java-11-openjdk-amd64
Default locale: en_US, platform encoding: UTF-8
OS name: "linux", version: "6.1.58+", arch: "amd64", family: "unix"



#### Download Apache Spark

NOTE: If spark is not downloaded. Let us make sure the version we are trying to download is officially supported at
https://spark.apache.org/downloads.html

In [ ]:
# Spark and Hadoop version
spark_version = 'spark-2.4.6'
hadoop_version = 'hadoop2.7'
spark_path = f"/opt/{spark_version}-bin-{hadoop_version}"
if not os.path.exists(spark_path):
  run(f"wget -q -nc -O apache-spark.tgz https://downloads.apache.org/spark/{spark_version}/{spark_version}-bin-{hadoop_version}.tgz")
  run('tar zxf apache-spark.tgz -C /opt')
  run('rm -f apache-spark.tgz')

os.environ["SPARK_HOME"] = spark_path
os.environ["PATH"] += ":$SPARK_HOME/bin"


>> wget -q -nc -O apache-spark.tgz https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz

>> tar zxf apache-spark.tgz -C /opt

>> rm -f apache-spark.tgz



#### Get Apache SystemDS

In [ ]:
!git clone https://github.com/apache/systemds systemds
%cd systemds

Cloning into 'systemds'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 155638 (delta 0), reused 1 (delta 0), pack-reused 155634
Receiving objects: 100% (155638/155638), 226.28 MiB | 22.37 MiB/s, done.
Resolving deltas: 100% (100687/100687), done.
/content/systemds


In [ ]:
# Logging flags: -q only for ERROR; -X for DEBUG; -e for ERROR
maven('clean package -P distribution')

>> /opt/apache-maven-3.6.3/bin/mvn clean package -q



In [ ]:
# Example Classification task
# !$SPARK_HOME/bin/spark-submit ./target/SystemDS.jar -f ./scripts/nn/examples/fm-binclass-dummy-data.dml

#### Playground for DML

The following code cell is for dml code.

In [ ]:
%%writefile /content/test.dml

# This code code acts as a playground for dml code
X = rand (rows = 20, cols = 10)
y = X %*% rand(rows = ncol(X), cols = 1)
lm(X = X, y = y)

Writing /content/test.dml


Run `dml` with Spark backend

In [ ]:
!$SPARK_HOME/bin/spark-submit \
    ./target/SystemDS.jar -f /content/test.dml


20/07/22 21:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
log4j:WARN No appenders could be found for logger (org.apache.sysds.api.DMLScript).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
ANTLR Tool version 4.5.3 used for code generation does not match the current runtime version 4.7ANTLR Runtime version 4.5.3 used for parser compilation does not match the current runtime version 4.7ANTLR Tool version 4.5.3 used for code generation does not match the current runtime version 4.7ANTLR Runtime version 4.5.3 used for parser compilation does not match the current runtime version 4.7Calling the Direct Solver...
Computing the statistics...
AVG_TOT_Y, 2.1068023514774974
STDEV_TOT_Y, 0.6111885165234278
AVG_RES_Y, 4.0434250725418505E-9
STDEV_RES_Y, 2.0038801747418575E-8
DISPERSION, 3.9409679055963126E-16
R2, 0.999999999

## Working with ONNX import

Import ONNX graph to dml based scripts.

In [ ]:
!pip install py4j

     |████████████████████████████████| 204kB 2.7MB/s 


In [ ]:
!pip install onnx

     |████████████████████████████████| 7.4MB 2.8MB/s 


In [ ]:
!python tests/onnx_systemds/test_models/model_generate.py

In [ ]:
%cd src/main/python
#!ls
#!python3 ./create_python_dist.py
#!cat tests/onnx/test_models/test_simple.TestSimpleOperators
!python -m systemds.onnx_systemds.convert tests/onnx_systemds/test_models/simple_if_graph.onnx

/content/systemds/src/main/python
ERROR environment variable SYSTEMDS_ROOT_PATH not set could not resolve path to module


In [ ]:
!cat simple_mat_add.dml


### This file was generated by onnx-systemds

# -------------------------------------------------------------
# Model
#  ir_version: 7
#  producer_name: onnx-systemds test-graph generator
#  model_version: 0
# opset_import:
#  - domain/opset = ONNX/12
# -------------------------------------------------------------



# Doc string of a simple matrix addition test graph
# --------------------------------------------------------------------------------------------
# INPUT PARAMETERS:
# --------------------------------------------------------------------------------------------
# NAME            TYPE     VALUE_TYPE   SHAPE    MEANING   
# A               matrix   double       [2, 2]   This is a description of variable A
# B               matrix   double       [2, 2]   This is a description of variable B
# D               matrix   double       [2, 2]   This is a description of variable D
# ---------------------------------------------------------------------------------------------
# OUTPU

### Working with SystemDS **Standalone**

(NOTE: Pay attention to *directories* and *relative paths*. :))

##### 1. Set SystemDS environement variables

These are useful for the `./bin/systemds` script.

In [3]:
%cd /content/

/content


In [5]:
!tar -xzf systemds-3.2.0-src.tgz

In [6]:
%ls

sample_data/  systemds-3.2.0-src/  systemds-3.2.0-src.tgz


In [7]:
%cd systemds-3.2.0-src/

/content/systemds-3.2.0-src


In [8]:
%ls

bin/      CONTRIBUTING.md  docker/  pom.xml                release.properties
CITATION  dev/             LICENSE  pom.xml.releaseBackup  scripts/
conf/     doap.rdf         NOTICE   README.md              src/


In [9]:
maven('clean package -P distribution')

Streaming output truncated to the last 5000 lines.
Downloaded from central: https://repo1.maven.org/maven2/org/apache/kerby/kerb-client/1.0.1/kerb-client-1.0.1.jar (113 kB at 19 kB/s)
Downloaded from central: https://repo1.maven.org/maven2/org/apache/kerby/kerb-simplekdc/1.0.1/kerb-simplekdc-1.0.1.jar (20 kB at 3.4 kB/s)
Downloaded from central: https://repo1.maven.org/maven2/org/apache/commons/commons-configuration2/2.8.0/commons-configuration2-2.8.0.jar (633 kB at 105 kB/s)
Downloaded from central: https://repo1.maven.org/maven2/org/apache/kerby/kerb-common/1.0.1/kerb-common-1.0.1.jar (65 kB at 11 kB/s)
Downloaded from central: https://repo1.maven.org/maven2/com/nimbusds/nimbus-jose-jwt/9.8.1/nimbus-jose-jwt-9.8.1.jar (444 kB at 74 kB/s)
Downloaded from central: https://repo1.maven.org/maven2/org/apache/kerby/token-provider/1.0.1/token-provider-1.0.1.jar (19 kB at 3.1 kB/s)
Downloaded from central: https://repo1.maven.org/maven2/org/apache/kerby/kerby-config/1.0.1/kerby-config-1.0.1.

In [10]:
!echo $(pwd)

/content/systemds-3.2.0-src


In [11]:
!export SYSTEMDS_ROOT=$(pwd)
!export SYSTEMDS_ROOT_PATH=$(pwd)
!export PATH=$SYSTEMDS_ROOT/bin:$PATH

In [12]:
!echo 'export SYSTEMDS_ROOT='$(pwd) >> ~/.bashrc
!echo 'export SYSTEMDS_ROOT_PATH='$(pwd) >> ~/.bashrc
!echo 'export PATH=$SYSTEMDS_ROOT/bin:$PATH' >> ~/.bashrc

##### 2. Download Haberman data

In [13]:
!mkdir ../data

In [14]:
!wget -P ../data/ http://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data

--2024-02-12 17:11:03--  http://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘../data/haberman.data’

haberman.data           [ <=>                ]   3.03K  --.-KB/s    in 0s      

2024-02-12 17:11:03 (232 MB/s) - ‘../data/haberman.data’ saved [3103]



##### 2.1 Set `metadata` for the data

In [15]:
# generate metadata file for the dataset
!echo '{"rows": 306, "cols": 4, "format": "csv"}' > ../data/haberman.data.mtd

# generate type description for the data
!echo '1,1,1,2' > ../data/types.csv
!echo '{"rows": 1, "cols": 4, "format": "csv"}' > ../data/types.csv.mtd

##### 3. Find the algorithm to run with `systemds`

In [16]:
# Inspect the directory structure
!ls

bin	  CONTRIBUTING.md	      doap.rdf	NOTICE		       README.md	   scripts
CITATION  dependency-reduced-pom.xml  docker	pom.xml		       release.properties  src
conf	  dev			      LICENSE	pom.xml.releaseBackup  scratch_space	   target


In [17]:
# List all the scripts
!ls scripts/algorithms

ALS-CG.dml	      CsplineDS.dml		 l2-svm-predict.dml	  random-forest.dml
ALS-DS.dml	      decision-tree.dml		 LinearRegCG.dml	  random-forest-predict.dml
ALS_predict.dml       decision-tree-predict.dml  LinearRegDS.dml	  StepGLM.dml
ALS_topk_predict.dml  GLM.dml			 m-svm.dml		  StepLinearRegDS.dml
apply-transform.dml   GLM-predict.dml		 m-svm-predict.dml	  stratstats.dml
bivar-stats.dml       KM.dml			 MultiLogReg.dml	  transform.dml
Cox.dml		      Kmeans.dml		 naive-bayes.dml	  Univar-Stats.dml
Cox-predict.dml       Kmeans-predict.dml	 naive-bayes-predict.dml
CsplineCG.dml	      l2-svm.dml		 PCA.dml


In [18]:
!./bin/systemds ./scripts/algorithms/Univar-Stats.dml -nvargs X=../data/haberman.data TYPES=../data/types.csv STATS=../data/univarOut.mtx CONSOLE_OUTPUT=TRUE

SYSTEMDS_ROOT not set defaulting to current dir /content/systemds-3.2.0-src
realpath: '': No such file or directory
realpath: '': No such file or directory
###############################################################################
#  SYSTEMDS_ROOT= .
#  SYSTEMDS_JAR_FILE= target/SystemDS.jar
#  SYSDS_EXEC_MODE= singlenode
#  CONFIG_FILE= -config ./src/test/config/SystemDS-config.xml
#  LOG4JPROP= ./src/test/resources/log4j.properties
#  CLASSPATH= target/SystemDS.jar:./lib/*:./target/lib/*
#  HADOOP_HOME= 
#
#  Running script ./scripts/algorithms/Univar-Stats.dml locally with opts: -nvargs X=../data/haberman.data TYPES=../data/types.csv STATS=../data/univarOut.mtx CONSOLE_OUTPUT=TRUE
###############################################################################
Executing command:     java -Xmx4g -Xms4g -Xmn400m    -cp target/SystemDS.jar:./lib/*:./target/lib/*   -Dlog4j.configuration=file:./src/test/resources/log4j.properties   org.apache.sysds.api.DMLScript   -f ./scripts/algori

##### 3.1 Let us inspect the output data

In [19]:
!cat ../data/univarOut.mtx

1 1 30.0
1 2 58.0
2 1 83.0
2 2 69.0
2 3 52.0
3 1 53.0
3 2 11.0
3 3 52.0
4 1 52.45751633986928
4 2 62.85294117647059
4 3 4.026143790849673
5 1 116.71458266366658
5 2 10.558630665380907
5 3 51.691117539912135
6 1 10.803452349303281
6 2 3.2494046632238507
6 3 7.189653506248555
7 1 0.6175922641866753
7 2 0.18575610076612029
7 3 0.41100513466216837
8 1 0.20594669940735139
8 2 0.051698529971741194
8 3 1.7857418611299172
9 1 0.1450718616532357
9 2 0.07798443581479181
9 3 2.954633471088322
10 1 -0.6150152487211726
10 2 -1.1324380182967442
10 3 11.425776549251449
11 1 0.13934809593495995
11 2 0.13934809593495995
11 3 0.13934809593495995
12 1 0.277810485320835
12 2 0.277810485320835
12 3 0.277810485320835
13 1 52.0
13 2 63.0
13 3 1.0
14 1 52.16013071895425
14 2 62.80392156862745
14 3 1.2483660130718954
15 4 2.0
16 4 1.0
17 4 1.0
